In [1]:
import import_ipynb
import sys
import torch
from torch import nn
from torch import optim

if torch.cuda.is_available():
    cuda = torch.cuda.device("cuda:0")

%cd /home/ankit/Commitments/earthquake-prediction/

from preprocessing import *
from training import *
from postprocessing import *
from testing import *

%cd /home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi/

/home/ankit/Commitments/earthquake-prediction
importing Jupyter notebook from preprocessing.ipynb
importing Jupyter notebook from training.ipynb
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 90])
importing Jupyter notebook from postprocessing.ipynb
importing Jupyter notebook from testing.ipynb
/home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi


In [2]:
# SETTINGS D

data_file = "data.csv"
time_parser = time_from_string_dmy
mag_range = (5, 6)

rows = 3
cols = 3
total_params = rows*cols
time_intervals = 690
train_size = 0.8
lookback_window = 50

In [3]:
earthquakes = extract_from_csv(data_file, time_parser, mag_range)
cells = Spatio_Temporal_Segmentation(rows, cols, time_intervals, earthquakes)

print(cells[0][0][0])

No earthquakes were recorded in this cell.


In [4]:
generator = TensorGen(cells)

eventtensor = generator.eventNoEvent()
freqtensor = generator.freq()
minmagtensor = generator.min_magnitude()
maxmagtensor = generator.max_magnitude()

if torch.cuda.is_available:
    eventtensor = eventtensor.cuda()
    freqtensor = freqtensor.cuda()
    minmagtensor = minmagtensor.cuda()
    maxmagtensor = maxmagtensor.cuda()
    
print(eventtensor.size())
print(freqtensor.size())
print(minmagtensor.size())
print(maxmagtensor.size())

torch.Size([690, 1, 9])
torch.Size([690, 1, 9])
torch.Size([690, 1, 9])
torch.Size([690, 1, 9])


<strong>TRAINING EVENT PREDICTOR</strong>

In [5]:
in_params = rows*cols*lookback_window
hidden1 = 200
hidden2 = 300
out_params = rows*cols

In [16]:
model = ANN(in_params, hidden1, hidden2, out_params)

if torch.cuda.is_available():
    model = model.cuda()
    
# out = model(eventtensor[:100])
# print(out)                    

In [17]:
# SETTINGS T1
pos_weight = torch.tensor([1.25])
pos_weight = pos_weight.cuda()
                          
loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
n_epochs = 100

In [28]:
optimizer = optim.Adagrad(model.parameters(), lr=0.001)

# tqdm._instances.clear()
train(model, eventtensor, optimizer, loss_function, n_epochs=1, lookback_window=lookback_window)

100%|██████████| 640/640 [00:00<00:00, 741.24it/s]

Epoch 1 completed. 
Training Loss: 0.594
Test Loss: 0.650


In [29]:
evaluate(data=eventtensor, model=model, train_size=0.8, lookback_window=lookback_window, threshold=0.5)

100%|██████████| 640/640 [00:00<00:00, 1232.05it/s]


Evaluation results for training dataset:
4518 evaluations in total.
1018/1971 cells with earthquake activity were correctly identified. (51.65 %)
2356/2547 cells with no earthquake activity were correctly identified. (92.50 %)
191/1209 positives turned out to be false.
953/3309 negatives turned out to be false.
Preicision: 0.842
Recall: 0.516
Overall accuracy: 74.68 %
Evaluation results for training dataset:
1242 evaluations in total.
214/493 cells with earthquake activity were correctly identified. (43.41 %)
669/749 cells with no earthquake activity were correctly identified. (89.32 %)
80/294 positives turned out to be false.
279/948 negatives turned out to be false.
Preicision: 0.728
Recall: 0.434
Overall accuracy: 71.10 %
